In [0]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import re

import numpy as np
from datetime import datetime

In [0]:
spark=SparkSession.builder.appName("AIS-Report").getOrCreate()
#spark
ps.set_option('compute.ops_on_diff_frames', True)

In [0]:

today_x=datetime.now()

In [0]:
main_cols=['id','incident_no','open_datetime','close_datetime','response_datetime','resolved_datetime', \
           'install_date','productype_name','brand_name','model_name','severity_name','incident_status_name','incident_type_name']

datetime_format='%d/%m/%Y %H:%M'
strDatetime_format='%d-%b-%y %H:%M'

In [0]:
# File uploaded to /FileStore/tables/incident_detail.csv
# File uploaded to /FileStore/tables/incident_master.csv

# Way1: Load by spark.sql
# load data by spark.sql
#incident_dfSpk=spark.read.csv("/FileStore/tables/incident_master.csv",header=True)
#incident_dfSpk.printSchema()
#detail_dfSpk=spark.read.csv("/FileStore/tables/incident_detail.csv",header=True)
#detail_dfSpk.printSchema()
# convert it to spark-pandas
#df_all=incident_dfSpk.to_pandas_on_spark()


# Way2: Load by pyspark.pandas
#df_all=ps.read_sql({sql},{connection}) 
df_all=ps.read_csv("/FileStore/tables/mini_incident_master.csv",)
print(df_all.info())



<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 44 entries, 0 to 43
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         44 non-null     int32 
 1   incident_no                44 non-null     object
 2   productype_name            44 non-null     object
 3   brand_name                 44 non-null     object
 4   model_name                 44 non-null     object
 5   incident_severity_id       44 non-null     int32 
 6   severity_name              44 non-null     object
 7   incident_type_id           44 non-null     int32 
 8   incident_type_name         44 non-null     object
 9   incident_status_id         44 non-null     int32 
 10  incident_status_name       44 non-null     object
 11  service_type_id            44 non-null     int32 
 12  service_type_name          44 non-null     object
 13  incident_customer_support  44 non-null     object
 14  serial_nu

In [0]:
#df_all[['open_datetime','response_datetime']].isna().sum()
df_all[main_cols].head(5)

,id,incident_no,open_datetime,close_datetime,response_datetime,resolved_datetime,install_date,productype_name,brand_name,model_name,severity_name,incident_status_name,incident_type_name
0,15,SR-ES-22-0015,3/3/2022 23:08,3/3/2022 23:35,3/3/2022 23:08,3/3/2022 23:35,None,Software,VMWare,vRealize Automation Advanced Per OSI,Minor,Closed,General Incident
1,16,SR-ES-22-0016,3/3/2022 17:35,3/3/2022 23:42,3/3/2022 17:35,3/3/2022 23:42,None,Software,VMWare,vRealize Automation Advanced Per OSI,Minor,Closed,General Incident
2,17,SR-ES-22-0017,3/3/2022 13:05,3/3/2022 16:42,3/3/2022 13:05,3/3/2022 16:42,None,Software,VMWare,vRealize Automation Advanced Per OSI,Minor,Closed,General Incident
3,18,SR-ES-22-0018,4/3/2022 11:21,4/3/2022 18:58,4/3/2022 11:21,4/3/2022 18:58,None,Software,VMWare,vRealize Automation Advanced Per OSI,Minor,Closed,General Incident
4,19,SR-ES-22-0019,8/3/2022 12:25,8/3/2022 17:08,8/3/2022 12:25,8/3/2022 17:08,None,Software,VMWare,vRealize Automation Advanced Per OSI,Minor,Closed,General Incident


In [0]:
df_all['open_datetime']=ps.to_datetime(df_all['open_datetime'], format=datetime_format)
df_all['close_datetime']=ps.to_datetime(df_all['close_datetime'], format=datetime_format)

df_all['response_datetime']=ps.to_datetime(df_all['response_datetime'], format=datetime_format)
df_all['resolved_datetime']=ps.to_datetime(df_all['resolved_datetime'], format=datetime_format)

In [0]:
df_all['install_date']=ps.to_datetime(df_all['install_date'], format=datetime_format)
df_all['eos_date']=ps.to_datetime(df_all['eos_date'], format=datetime_format)
#df_all[['open_datetime'	,'close_datetime',	'response_datetime','resolved_datetime','install_date','eos_date']]

In [0]:
# it has been supported yet in this version, but spark plan to implement further
# Convert datetime to find time delta  
def convert_datetime_to_timedelta(item):
    if item['resolved_datetime'] is not None:
        start = item['open_datetime']
        end = item['resolved_datetime']

        delta = end - start
        #print(type(delta))
        # str_delta = str(delta)
        
        return delta
    return None
  
  # we can implement like this
df_all['work_around_time_hours']=(df_all.resolved_datetime-df_all.open_datetime)/ (60*60)
df_all['work_around_time_hours']=df_all['work_around_time_hours'].round(1)

/databricks/spark/python/pyspark/pandas/data_type_ops/datetime_ops.py:61: UserWarning: Note that there is a behavior difference of timestamp subtraction. The timestamp subtraction returns an integer in seconds, whereas pandas returns 'timedelta64[ns]'.
  warnings.warn(msg, UserWarning)


In [0]:
df_all ['is_update_sw']=df_all['incident_type_id'].apply( lambda x : 'Update patch' if x==14 else '' )

In [0]:
df_all['month_year']= df_all['open_datetime'].apply(lambda x: x.strftime('%m-%Y')) 

In [0]:
def sla_in_out(item):
    # add service type= incident
    # critical and by 4 hour
    if item["incident_severity_id"]==1 :
       if item['work_around_time_hours']<=4:
        return "in"
       else:
        return "out"
        
    elif item["incident_severity_id"]==2:
       if item['work_around_time_hours']<=  168 :
        return "in"
       else:
        return "out"
    
    elif item["incident_severity_id"]==3:
       if item['work_around_time_hours']<=  (168 * 3) :
        return "in"
       else:
        return "out"
    else:
        return "cosmatic"
df_all['sla']=df_all.apply(sla_in_out,axis=1)  

In [0]:
def cal_againg(item):
    aging_year=0
    if  ps.isna(item["install_date"])==False  :
      aging_year= round( abs(today_x-item['install_date']).days/365,1)
      # print(f'{today_x} to {item["install_date"]}={aging_year}')
    else:
      aging_year= np.nan 
      # print (np.nan) 
    return aging_year

df_all['today']=today_x
df_all['aging_year']=df_all.apply( cal_againg,axis=1)

In [0]:
# load detail
df_detail=ps.read_csv("/FileStore/tables/mini_incident_detail.csv")
masterIDs=df_all['id'].tolist()
df_detail=ps.sql("select * from {df_detail} WHERE incident_id	IN {masterIDs}")
print(df_detail.info())


<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 50 non-null     int32 
 1   incident_id        50 non-null     int32 
 2   task_start         50 non-null     object
 3   task_end           48 non-null     object
 4   case_id            9 non-null      object
 5   employee_id        50 non-null     int32 
 6   service_team_name  50 non-null     object
 7   engineer_name      50 non-null     object
dtypes: int32(3), object(5)None


In [0]:
def all_detail_by_incident(row):
    
    master_id = row['id']
    dfx =ps.sql("select * from {df_detail} where incident_id={master_id}")
    des = ''
    if df_detail.empty == False:
        for index, detail in dfx.iterrows():
            x = f"{detail['service_team_name']} | {detail['engineer_name']} | {detail['task_start']} - {detail['task_end']} \n"
            des = des + x

    return des

df_all['detail'] = df_all.apply(all_detail_by_incident, axis=1)


In [0]:

sql_count_eng="select incident_id , count(distinct employee_id) as no_engineer from {df_detail} where incident_id in {masterIDs}  group by  incident_id  "
dfDetail_noEng=ps.sql(sql_count_eng)
df_all=df_all.merge(dfDetail_noEng,how='left',left_on='id',right_on='incident_id')

In [0]:
#df_detail.groupby(['incident_id'], as_index=False).agg({'case_id' : ','.join  })
#aggs must be a dict mapping from column name to aggregate functions (string or list of strings).


In [0]:
df_all=df_all.sort_values(by=['open_datetime'],ascending=True)

In [0]:
df_all['open_datetime']=df_all['open_datetime'].dt.strftime('%d-%b-%y %H:%M')
df_all['close_datetime']=df_all['close_datetime'].dt.strftime('%d-%b-%y %H:%M')
df_all['response_datetime']=df_all['response_datetime'].dt.strftime('%d-%b-%y %H:%M')
df_all['resolved_datetime']=df_all['resolved_datetime'].dt.strftime('%d-%b-%y %H:%M')
df_all['install_date']=df_all['install_date'].dt.strftime('%d-%b-%y %H:%M')
df_all['eos_date']=df_all['eos_date'].dt.strftime('%d-%b-%y %H:%M')

In [0]:
df_all

,id,incident_no,productype_name,brand_name,model_name,incident_severity_id,severity_name,incident_type_id,incident_type_name,incident_status_id,incident_status_name,service_type_id,service_type_name,incident_customer_support,serial_number,datacenter_name,open_datetime,close_datetime,response_datetime,resolved_datetime,install_date,eos_date,customer_warranty_start,customer_warranty_end,customer_support,failure_type,work_around_time_hours,is_update_sw,month_year,sla,today,aging_year,detail,incident_id,no_engineer
17,42,SR-ES-22-0042,Software,VMWare,vRealize Automation Advanced Per OSI,3,Minor,1,General Incident,4,Closed,2,Request,Kittisak Punsukphol <kittispu@ais.co.th>,-,SILA1,05-Jan-22 00:01,05-Jan-22 04:00,05-Jan-22 00:01,05-Jan-22 04:00,None,01-Aug-18 00:00,30/7/2021 0:00,29/7/2022 0:00,Anan Sasithonchaidech,None,4.0,,01-2022,in,2022-07-28 11:22:13.934003,NaN,Onsite | Pimonkit Pimonkittikul | 5/1/2022 0:0...,42,1
18,43,SR-ES-22-0043,Software,VMWare,vRealize Automation Advanced Per OSI,3,Minor,1,General Incident,4,Closed,1,Incident,Jariya Moongkheemklang <jariyam@ais.co.th>,-,SILA1,05-Jan-22 13:41,06-Jan-22 14:49,05-Jan-22 13:41,06-Jan-22 14:49,None,01-Aug-18 00:00,30/7/2021 0:00,29/7/2022 0:00,Anan Sasithonchaidech,None,25.1,,01-2022,in,2022-07-28 11:22:13.934003,NaN,Onsite | Pimonkit Pimonkittikul | 5/1/2022 16:...,43,1
21,46,SR-ES-22-0046,Software,VMWare,vRealize Automation Advanced Per OSI,3,Minor,1,General Incident,4,Closed,1,Incident,Jariya Moongkheemklang <jariyam@ais.co.th>,-,SILA1,10-Jan-22 14:50,13-Jan-22 10:10,10-Jan-22 14:50,10-Jan-22 16:34,None,01-Aug-18 00:00,30/7/2021 0:00,29/7/2022 0:00,Anan Sasithonchaidech,None,1.7,,01-2022,in,2022-07-28 11:22:13.934003,NaN,Onsite | Pimonkit Pimonkittikul | 10/1/2022 14...,46,1
19,44,SR-ES-22-0044,Software,VMWare,vRealize Automation Advanced Per OSI,3,Minor,1,General Incident,4,Closed,1,Incident,Jariya Moongkheemklang <jariyam@ais.co.th>,-,SILA1,11-Jan-22 15:25,11-Jan-22 18:48,11-Jan-22 15:25,11-Jan-22 15:56,None,01-Aug-18 00:00,30/7/2021 0:00,29/7/2022 0:00,Anan Sasithonchaidech,None,0.5,,01-2022,in,2022-07-28 11:22:13.934003,NaN,Onsite | Pimonkit Pimonkittikul | 11/1/2022 15...,44,1
22,47,SR-ES-22-0047,Software,VMWare,vRealize Automation Advanced Per OSI,3,Minor,1,General Incident,4,Closed,1,Incident,Jariya Moongkheemklang <jariyam@ais.co.th>,-,SILA1,13-Jan-22 15:37,13-Jan-22 18:44,13-Jan-22 15:37,13-Jan-22 18:44,None,01-Aug-18 00:00,30/7/2021 0:00,29/7/2022 0:00,Anan Sasithonchaidech,None,3.1,,01-2022,in,2022-07-28 11:22:13.934003,NaN,Onsite | Pimonkit Pimonkittikul | 13/1/2022 15...,47,1
36,66,SR-ES-22-0066,Software,VMWare,vRealize Automation Advanced Per OSI,3,Minor,1,General Incident,4,Closed,1,Incident,Worrawat Narongkhachavana <worrawan@ais.co.th>,-,SILA1,15-Jan-22 16:26,15-Feb-22 18:30,15-Feb-22 16:33,15-Feb-22 18:30,None,01-Aug-18 00:00,30/7/2021 0:00,29/7/2022 0:00,Anan Sasithonchaidech,None,746.1,,01-2022,out,2022-07-28 11:22:13.934003,NaN,Onsite | Pimonkit Pimonkittikul | 15/2/2022 16...,66,1
23,48,SR-ES-22-0048,Software,VMWare,vRealize Automation Advanced Per OSI,3,Minor,1,General Incident,4,Closed,1,Incident,Jariya Moongkheemklang <jariyam@ais.co.th>,-,SILA1,28-Jan-22 11:18,28-Jan-22 13:29,28-Jan-22 11:39,28-Jan-22 13:29,None,01-Aug-18 00:00,30/7/2021 0:00,29/7/2022 0:00,Anan Sasithonchaidech,None,2.2,,01-2022,in,2022-07-28 11:22:13.934003,NaN,Onsite | Pimonkit Pimonkittikul | 28/1/2022 11...,48,1
24,49,SR-ES-22-0049,Software,VMWare,vRealize Automation Advanced Per OSI,3,Minor,1,General Incident,4,Closed,1,Incident,Pratya Prechachan <pratyapr@ais.co.th>,-,SILA1,31-Jan-22 09:50,31-Jan-22 16:40,31-Jan-22 10:22,31-Jan-22 16:40,None,01-Aug-18 00:00,30/7/2021 0:00,29/7/2022 0:00,Anan Sasithonchaidech,None,6.8,,01-2022,in,2022-07-28 11:22:13.934003,NaN,Onsite | Pimonkit Pimonkittikul | 31/1/2022 10...,49,1
26,51,SR-ES-22-0051,Software,VMWare,vRealize Automation Advanced Per OSI,3,Minor,1,General Incident,4,Closed,1,Incident,Jariya Moongkheemklang <jariyam@ais.co.th>